In [ ]:
#This is for map insurence hover india state wise data
import pandas as pd
import json
import os
#This is to direct the path to get the data as states
path=r"D:/BONEYS/WEB/PYTHON/Project/PhonePe/data/map/insurance/hover/country/india/state"
Map_Insu_Hover_state_list=os.listdir(path)
#This is to extract the data's to create a dataframe
clm={'State':[], 'Year':[],'Quater':[],'Insurance':[], 'Insurance_count':[], 'Insurance_amount':[]}
for i in Map_Insu_Hover_state_list:
    p_i=path+"/"+i
    Agg_yr=os.listdir(p_i)
    #delhi
    for j in Agg_yr:
        p_j=p_i+"/"+j
        #2018
        Agg_yr_list=os.listdir(p_j)
        for k in Agg_yr_list:
            p_k=p_j+"/"+k 
            #1.json
            Data=open(p_k,'r')
            D=json.load(Data)
            for z in D['data']['hoverDataList']:
              Name=z['name']
              count=z['metric'][0]['count']
              amount=z['metric'][0]['amount']
              clm['Insurance'].append(Name)
              clm['Insurance_count'].append(count)
              clm['Insurance_amount'].append(amount)
              clm['State'].append(i)
              clm['Year'].append(j)
              clm['Quater'].append(int(k.strip('.json')))
#Succesfully created a dataframe
Map_Insu_state=pd.DataFrame(clm)
print(Map_Insu_state)

                           State  Year  Quater  \
0      andaman-&-nicobar-islands  2020       2   
1      andaman-&-nicobar-islands  2020       2   
2      andaman-&-nicobar-islands  2020       3   
3      andaman-&-nicobar-islands  2020       3   
4      andaman-&-nicobar-islands  2020       3   
...                          ...   ...     ...   
13871                west-bengal  2024       4   
13872                west-bengal  2024       4   
13873                west-bengal  2024       4   
13874                west-bengal  2024       4   
13875                west-bengal  2024       4   

                               Insurance  Insurance_count  Insurance_amount  
0                 south andaman district                3             795.0  
1                      nicobars district                3             565.0  
2      north and middle andaman district                1             281.0  
3                 south andaman district               35           13651.0  
4        

In [4]:
#You need to map your State names to the names used in the GeoJSON file (properties.ST_NM). Otherwise you will get blank figure.
state_name_map = {
    'andaman-&-nicobar-islands': 'Andaman & Nicobar Island',
    'andhra-pradesh': 'Andhra Pradesh',
    'arunachal-pradesh': 'Arunanchal Pradesh',
    'assam': 'Assam',
    'bihar': 'Bihar',
    'chandigarh': 'Chandigarh',
    'chhattisgarh': 'Chhattisgarh',
    'dadra-&-nagar-haveli-&-daman-&-diu': 'Dadra and Nagar Haveli',
    'delhi': 'NCT of Delhi',
    'goa': 'Goa',
    'gujarat': 'Gujarat',
    'haryana': 'Haryana',
    'himachal-pradesh': 'Himachal Pradesh',
    'jammu-&-kashmir': 'Jammu & Kashmir',
    'jharkhand': 'Jharkhand',
    'karnataka': 'Karnataka',
    'kerala': 'Kerala',
    'ladakh': 'Ladakh',
    'lakshadweep': 'Lakshadweep',
    'madhya-pradesh': 'Madhya Pradesh',
    'maharashtra': 'Maharashtra',
    'manipur': 'Manipur',
    'meghalaya': 'Meghalaya',
    'mizoram': 'Mizoram',
    'nagaland': 'Nagaland',
    'odisha': 'Odisha',
    'puducherry': 'Puducherry',
    'punjab': 'Punjab',
    'rajasthan': 'Rajasthan',
    'sikkim': 'Sikkim',
    'tamil-nadu': 'Tamil Nadu',
    'telangana': 'Telangana',
    'tripura': 'Tripura',
    'uttar-pradesh': 'Uttar Pradesh',
    'uttarakhand': 'Uttarakhand',
    'west-bengal': 'West Bengal'
}
Map_Insu_state['State'] = Map_Insu_state['State'].str.strip().str.lower()
Map_Insu_state['State'] = Map_Insu_state['State'].replace(state_name_map)

In [ ]:
#creating table to store the above data
import mysql.connector
connection = mysql.connector.connect(
    host="localhost",
    user="xxx", #give your user name. Mostly its root
    password="xxx", #give your password
    database="xxx" #give any database name you want
)
cursor=connection.cursor()
query1 = """
        create table Map_Insurance_State(
        State varchar(100),
        Year int,
        Quater int,
        Insurance_Type varchar(100),
        Insurance_count float,
        Insurance_amount float)
        """
cursor.execute(query1)

In [4]:
#storing the above in mysql database
#all data must be tuple and must be a list of tuples
data=[]
for index in Map_Insu_state.index:
    row = Map_Insu_state.loc[index]
    row = row.values
    row = row[0],int(row[1]),int(row[2]),row[3],int(row[4]),float(row[5])
    data.append(row)
query2 = "insert into Map_Insurance_State values (%s,%s,%s,%s,%s,%s)"
cursor.executemany(query2,data)
connection.commit()

In [7]:
#doing data cleaning for Map_Insu_state
from sklearn.preprocessing import LabelEncoder
#first step check and remove null values
Map_Insu_state.isna().sum()
Map_Insu_state.info()
#Second do encoding and dummies to make all values int
enc_col = ['Insurance']
dummy_col = ['Year','State','Quater']
encoder = LabelEncoder()
Map_Insu_state['encoded_insurance'] = encoder.fit_transform(Map_Insu_state['Insurance'])
Map_Insu_state = pd.get_dummies(Map_Insu_state,columns=dummy_col,dtype=int)
#third removing outliners
Map_Insu_state['insurance_count_std']=(Map_Insu_state['Insurance_count']-Map_Insu_state['Insurance_count'].mean())/Map_Insu_state['Insurance_count'].std()
Map_Insu_state['insurance_amount_std']=(Map_Insu_state['Insurance_amount']-Map_Insu_state['Insurance_amount'].mean())/Map_Insu_state['Insurance_amount'].std()
condition1 = Map_Insu_state['insurance_count_std']<=3
condition2 = Map_Insu_state['insurance_count_std']>=-3
condition3 = Map_Insu_state['insurance_amount_std']<=3
condition4 = Map_Insu_state['insurance_amount_std']>=-3
Map_Insu_state = Map_Insu_state[(condition1) & (condition2) & (condition3) & (condition4)]
print(Map_Insu_state)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13876 entries, 0 to 13875
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   State             13876 non-null  object 
 1   Year              13876 non-null  object 
 2   Quater            13876 non-null  int64  
 3   Insurance         13876 non-null  object 
 4   Insurance_count   13876 non-null  int64  
 5   Insurance_amount  13876 non-null  float64
dtypes: float64(1), int64(2), object(3)
memory usage: 650.6+ KB
                               Insurance  Insurance_count  Insurance_amount  \
0                 south andaman district                3             795.0   
1                      nicobars district                3             565.0   
2      north and middle andaman district                1             281.0   
3                 south andaman district               35           13651.0   
4                      nicobars district                5

In [6]:
#plotting aggregated_state_insurance in the form of india map using plotly
import plotly.express as px

fig = px.choropleth(
    Map_Insu_state,
    geojson="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson",
    featureidkey='properties.ST_NM',
    locations='State',
    color='Insurance_count',
    color_continuous_scale='Reds'
)

fig.update_geos(fitbounds="locations", visible=False)

fig.show()

In [ ]:
#This is for map insurence hover india 2020 data
import pandas as pd
import json
import os

#This is to direct the path to get the data as states

path=r"D:/BONEYS/WEB/PYTHON/Project/PhonePe/data/map/insurance/hover/country/india/2020"
Map_Hover_2020_list=os.listdir(path)

#This is to extract the data's to create a dataframe
clm={'Quater':[],'Insurance':[], 'Insurance_count':[], 'Insurance_amount':[]}

for i in Map_Hover_2020_list:
    p_i=path+"/"+i
    Data=open(p_i,'r')
    D=json.load(Data)
    for z in D['data']['hoverDataList']:
        Name=z['name']
        count=z['metric'][0]['count']
        amount=z['metric'][0]['amount']
        clm['Insurance'].append(Name)
        clm['Insurance_count'].append(count)
        clm['Insurance_amount'].append(amount)
        clm['Quater'].append(int(i.strip('.json')))
#Succesfully created a dataframe
Map_Insu_2020=pd.DataFrame(clm)
print(Map_Insu_2020)

     Quater          Insurance  Insurance_count  Insurance_amount
0         2         puducherry              112           22251.0
1         2         tamil nadu             5473         1075552.0
2         2      uttar pradesh             9884         1912266.0
3         2     madhya pradesh             6283         1198701.0
4         2     andhra pradesh            22104         3982391.0
..      ...                ...              ...               ...
101       4    jammu & kashmir              647          734861.0
102       4                goa              614          682364.0
103       4  arunachal pradesh              125           84247.0
104       4              delhi             9725         8045442.0
105       4        uttarakhand             1756         1961891.0

[106 rows x 4 columns]
